In [1]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait
from kdags.assets.reparation.reso.web_driver import initialize_driver, login_to_reso
from kdags.assets.reparation.reso.document_extractor import (
    extract_docs_data,
    click_see_service_order,
)
from kdags.assets.reparation.reso.file_processor import process_docs_data
from kdags.assets.reparation.reso.web_utils import wait_for_page_load, ensure_clickable
from kdags.assets.reparation.reso.file_utils import save_file_locally
from kdags.assets.reparation.reso.config import DEFAULT_WAIT
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)

In [2]:
login_to_reso(driver, wait)

True

In [3]:
service_order_df = pd.read_excel(
    r"C:\Users\andmn\PycharmProjects\kbooks\para_descargar.xlsx"
)
service_order_df

,service_order,equipment_name,customer_work_order,component_serial,changeout_date,subcomponent_name
0,14140838,TK399,18123419,EE13070408,2023-10-30,alternador_principal
1,14141425,TK859,18123759,EE13090440,2023-11-08,alternador_principal
2,14140979,TK883,18123800,EE14010717,2023-12-01,alternador_principal
3,14142021,TK852,18123924,EE19090298,2023-12-09,alternador_principal
4,14143179,TK868,18125034,EE14090173,2024-02-07,alternador_principal
5,14143655,TK851,18125407,EE14070623,2024-03-14,alternador_principal
6,14144274,TK849,18125653,EE14030724,2024-03-28,alternador_principal
7,14146810,TK860,18127630,EE11060983A,2024-04-03,alternador_principal
8,14144747,TK285,18126171,EE04090417,2024-04-10,alternador_principal
9,14144858,TK848,18126402,EE08080858,2024-04-24,alternador_principal


In [8]:
# Ensure the 'processed' column exists (default False)
if "processed" not in service_order_df.columns:
    service_order_df["processed"] = False


# Iterate over service orders that are not yet processed
for idx, ir in service_order_df.iterrows():
    if ir.processed:
        continue  # Skip already processed rows

    print(f"\nProcessing Row {idx} for Service Order: {ir.service_order}")

    # Wait for page to be fully loaded
    wait_for_page_load(wait)

    # Wait for and enter the service order code
    input_field = wait.until(EC.presence_of_element_located((By.ID, "codigoBuscar")))
    input_field.clear()
    input_field.send_keys(ir.service_order)

    # Click search button
    search_button = wait.until(EC.element_to_be_clickable((By.ID, "buttonBuscar")))
    ensure_clickable(driver, search_button)
    search_button.click()

    # Wait for results to load
    wait_for_page_load(wait)
    wait.until(EC.presence_of_element_located((By.ID, "WebGrid")))

    # Click the 'See service order' button
    click_see_service_order(wait)

    # Wait for page load after clicking service order
    wait_for_page_load(wait)

    # Click the Documents tab and wait for it to load
    docs_tab = wait.until(EC.element_to_be_clickable((By.ID, "v-pills-documentos-tab")))
    ensure_clickable(driver, docs_tab)
    docs_tab.click()
    wait_for_page_load(wait)

    # Extract the docs data from the documents section
    docs_data = extract_docs_data(driver, wait)

    # Skip processing if no documents were found
    if docs_data is None:
        print(
            f"No documents found for service order {ir.service_order} - marking as processed"
        )
        service_order_df.loc[idx, "processed"] = True
        continue

    # Process the extracted docs data
    processed_df = process_docs_data(
        docs_data, ir.subcomponent_name, ir.component_serial
    ).query("file_type=='INFORMES'")

    # Iterate over each document row and upload
    for jdx, row in processed_df.iterrows():
        result_url = save_file_locally(
            target_folder=row["target_folder"],
            file_name=row["file_name"],
            blob_url=row["blob_url"],
        )
        print(f"Document {jdx}: Uploaded successfully: {result_url}")

    # Click the close button
    close_button = wait.until(EC.element_to_be_clickable((By.ID, "closePCI")))
    ensure_clickable(driver, close_button)
    close_button.click()

    # Wait for page to return to initial state
    wait_for_page_load(wait)

    # Mark the service order as processed
    service_order_df.loc[idx, "processed"] = True


Processing Row 17 for Service Order: 14142811
Clicked 'See service order' button successfully.
Document 2: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\YK00487\INFORMES\14142166_292920_INI_APRO_Informe Tecnico Preliminar [Revisado].pdf
Document 5: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\YK00487\INFORMES\14142166_292920_FIN_APRO_Informe Tecnico Final [Revisado].pdf

Processing Row 18 for Service Order: 14142812
Clicked 'See service order' button successfully.
Document 2: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\WH340\INFORMES\14142811_294413_INI_APRO_Informe Tecnico Preliminar [Revisado].pdf
Document 4: Uploaded successfully: C:\Users\andmn\OneDrive - Komatsu Ltd\REPARACION\CILINDRO_DIRECCION\WH340\INFORMES\14142811_294413_FIN_APRO_Informe Tecnico Final [Revisado].pdf

Processing Row 19 for Service Order: 14141508
Clicked 'See service order'

In [11]:
service_order_df.subcomponent_name.unique()

array(['alternador_principal', 'cilindro_direccion', 'cilindro_levante',
       'motor_traccion', 'suspension_trasera'], dtype=object)

In [5]:
service_order_df

,service_order,equipment_name,customer_work_order,component_serial,changeout_date,subcomponent_name,processed
0,14140838,TK399,18123419,EE13070408,2023-10-30,alternador_principal,True
1,14141425,TK859,18123759,EE13090440,2023-11-08,alternador_principal,False
2,14140979,TK883,18123800,EE14010717,2023-12-01,alternador_principal,False
3,14142021,TK852,18123924,EE19090298,2023-12-09,alternador_principal,False
4,14143179,TK868,18125034,EE14090173,2024-02-07,alternador_principal,False
5,14143655,TK851,18125407,EE14070623,2024-03-14,alternador_principal,False
6,14144274,TK849,18125653,EE14030724,2024-03-28,alternador_principal,False
7,14146810,TK860,18127630,EE11060983A,2024-04-03,alternador_principal,False
8,14144747,TK285,18126171,EE04090417,2024-04-10,alternador_principal,False
9,14144858,TK848,18126402,EE08080858,2024-04-24,alternador_principal,False


In [17]:
processed_df.target_folder

1    01.%20%C3%81REAS%20KCH/1.6%20CONFIABILIDAD/CAE...
Name: target_folder, dtype: object